In [3]:
# Imports
import pandas as pd
import pronouncing
import re


# Read data from external mappings, and convert to Pandas DF.
pd.set_option('display.max_colwidth', 0)
consonants = pd.read_csv('consonants.csv')
vowels = pd.read_csv('vowels.csv')

def boldStr(str):
    """Bold the input string"""
    return '\033[1m'+str+'\033[0m'

def wordToPhonemeList(word):
    """Converts a single word to list of phonemes."""
    phoneme_str = pronouncing.phones_for_word(word)[0]
    phoneme_str_nostress = ''.join([i for i in phoneme_str if not i.isdigit()])
    return phoneme_str_nostress.split(' ')


def phonemeToDescription(phoneme, inventory):
    """Converts a single phoneme to its articulatory description."""
    return inventory[inventory['cmu_symbol']==phoneme]['description'].to_string(index=False).strip()


def wordToDescription(word):
    """Converts a single word to its articulatory description."""
    list_description = ''
    for phoneme in wordToPhonemeList(word):
        description = ''
        if consonants['cmu_symbol'].str.contains(phoneme).any():
            description = phonemeToDescription(phoneme, consonants)
        elif vowels['cmu_symbol'].str.contains(phoneme).any():
            description = phonemeToDescription(phoneme, vowels)
        list_description += ' ' + description
            
    return list_description.strip()


def docToDescription(doc):
    """Converts a whole document to its articulatory description."""
    doc_description = ''
    word_list = re.sub(r'[^\w\s\']', '', doc)
    for word in word_list.split(' '):
        doc_description += wordToDescription(word)
    return doc_description.strip()


def replaceTags(doc, name, pro, pro_pos):
    """Replaces tags like <PRO> with actual content."""
    # Basic pronoun swap.
    doc = re.sub(r'<PRO>', pro, doc)
    doc = re.sub(r'<PRO_POS>', pro_pos, doc)
    # Add name.
    doc = doc.replace(pro, name, 1)
    # Capitalize everything after end punctuation.
    punct_re = re.compile('([.!?]\s*)')
    split_on_punct = punct_re.split(doc)
    doc = ''.join([i.capitalize() for i in split_on_punct])
    return doc


def generate(scripts):
    """Given script data, generate a chapter."""
    
    # Start with an empty book.
    book = boldStr("SIMPLE DIALOGUES\n\n\n") + ("=" * 80) + "\n\n\n"
    chapter_counter = 0
    
    for script in scripts:
        # Construct chapter title.
        interior_or_exterior = scripts[chapter_counter]["title"][0]
        location_description = scripts[chapter_counter]["title"][1]
        location_time = scripts[chapter_counter]["title"][2]
        title = "\t%d. %s %s - %s\n\n" %(
            chapter_counter,
            interior_or_exterior,
            location_description,
            location_time
        )
        chapter = boldStr(title)

        # Construct chapter prologue.
        prologue = scripts[chapter_counter]["prologue"] + "\n\n"
        chapter += prologue

        # Construct chapter dialogue with generated content.
        for turn in scripts[chapter_counter]["dialogue"]:
            # Parse dialogue info.
            speaker = turn[0].upper() # The speaker's name, e.g. Raymond
            pronoun = turn[1] # The speaker's pronoun, e.g. he
            pronoun_possessive = turn[2] # The speaker's possessive pronoun, e.g. his
            turn_simple = turn[3] # The actual spoken content, e.g. "Coffee please."

            # Expand the dialogue turn.
            turn_expanded = docToDescription(turn_simple) # Generate pronunciation description.

            # Spoken words.
            spoken = "  \"%s\"" %(turn_simple) # The actual spoken content, with a little formatting.

            # Clean up the generated text by replacing pronoun tags with pronouns.
            turn_final = replaceTags(
                turn_expanded,
                speaker,
                pronoun,
                pronoun_possessive
            ) + "\n\n\t" + boldStr(speaker.upper()) + ":" + spoken + "\n\n\n\n"
            chapter += turn_final

        # Construct chapter epilogue.
        epilogue = scripts[chapter_counter]["epilogue"]
        chapter += epilogue

        # Add completed chapter to book.
        book += chapter + "\n\n\n\n\n\n" + ("=" * 100) + "\n\n\n"
        chapter_counter += 1
    
    return book

In [4]:
scripts = [
    {
        "title" : ["INT.","Mud Spot", "7:00AM"],
        "prologue" : "Raymond approaches the counter.",
        "dialogue" : [
            ["Raymond", "he", "his", "Small drip please."],
            ["Barista", "she", "her", "Cream or sugar?"],
            ["Raymond", "he", "his",  "Small drip please."],
            ["Barista", "she", "her",  "Your name?"],
            ["Raymond", "he", "his",  "Raymond."],
        ],
        "epilogue" : "The barista nods, reaches for a cup, and begins his order."
    },
    {
        "title" : ["EXT.","2nd and 12th", "7:15AM"],
        "prologue" : "Raymond looks right, and steps into the street.",
        "dialogue" : [
            ["Bicyclist", "he", "his", "Hey!"],
        ],
        "epilogue" : "Raymond steps back and sips his coffee. His jacket flaps once in the bicyclist's draft."
    },
    {
        "title" : ["INT.","Home", "10:00AM"],
        "prologue" : "Raymond's phone buzzes. He lets it go to voicemail.",
        "dialogue" : [
            ["Raymond's mom", "she", "her", "Hi honey. Give me a call back, okay?"],
        ],
        "epilogue" : "She hangs up."
    },
    {
        "title" : ["EXT.","Summersville Farmer’s Market", "10:10AM"],
        "prologue" : "Raymond takes a sip of his coffee, walking through the market stands",
        "dialogue" : [
            ["Farmer Guy", "he", "his", "Hey! Would you like to try some fungi?"],
            ["Raymond", "he", "his", "What are these?"],
            ["Farmer Guy", "he", "his", "These are called Chicken of the woods?"],
            ["Raymond", "he", "his",  "Chicken?"],
            ["Farmer Guy", "he", "his", "Yes! Because they kinda taste like chicken."],
            ["Raymond", "he", "his",  "Sounds interesting."],
            ["Farmer Guy", "he", "his", "Sauté some with butter. It's amazing."],
            ["Raymond", "he", "his",  "Alright. Um... I shall try some."],
            ["Farmer Guy", "he", "his", "Here you go."],
          ],
        "epilogue" : "Raymond gives cash to the guy, exchanges a smile then walks away."
    },
     {
        "title" : ["EXT.","Summersville Park", "11:30AM"],
        "prologue" : "Raymond sits on the bench as an medium-sized Australian Shepherd approaches him.",
        "dialogue" : [
            ["Raymond", "he", "his", "Hey buddy!"],
            ["Dog", "It", "its", "ah uh ah uh"],
            ["Raymond", "he", "his", "Yes? You are asking about the weather?"],
            ["Dog", "It", "its", "roooff...ooofoff"],
            ["Raymond", "he", "his", "Sit"],
            ["Dog", "It", "its", " "],
            ["Raymond", "he", "his", "Aw. You are such a good boy"],
            ["A woman", "she", "her", "Raymond! Raymond! Come here. Come back here."],
            ["Raymond", "he", "his", "Oh, off you go I guess. Wait, your name is Raymond, too?"],
          ],
        "epilogue" : "Raymond watches Raymond running back to the woman."
    },
     {
        "title" : ["INT.","Home", "1:00PM"],
        "prologue" : "Raymond's phone starts ringing.",
        "dialogue" : [
            ["Raymond", "he", "his", "Hello."],
            ["Delivery Person", "they", "their", "Hello? Can you hear me?"],
            ["Raymond", "he", "his", "Yes, I can hear you. Who is this?"],
            ["Delivery Person", "they", "their", "Is this Raymond D? I have a package for you to sign."],
            ["Raymond", "he", "his", "Yes, it's me"],
            ["Delivery Person", "they", "their", "I tried to ring your doorbell. It didn't work."],
            ["Raymond", "he", "his", "Oh. That's weird. I will come down."],
            ["Delivery Person", "they", "their", "Thank you."],
        
        ],
        "epilogue" : "Raymond runs out of the door."
    },
     {
        "title" : ["INT.","Home", "2:00PM"],
        "prologue" : "Raymond's takes out his phone and dials.",
        "dialogue" : [
            ["Raymond", "he", "his",  "Hey mom. Just checking in. Missed your call ealier. Call me back."],
        ],
        "epilogue" : "She hangs up."
    },
    {
        "title" : ["INT.","Home", "3:30PM"],
        "prologue" : "Raymond opens his laptop and joins a virtual call.",
        "dialogue" : [
            ["Steve", "he", "his",  "How's it going, everyone?"],
            ["Raymond", "he", "his",  "Doing great."],
            ["Linda", "she", "her",  "You know. Doing alright"],
            ["Kai", "he", "his",  "Oh sorry, muted. Hi."],
            ["Steve", "he", "his",  "Sorry for the last-minute notice. I just wanted to share the news."],
            ["Linda", "she", "her",  "News? Did they approve it?"],
            ["Steve", "he", "his",  "Yes. Yes, they did."],
            ["Kai", "he", "his",  "Oh my god."],
            ["Raymond", "he", "his",  "That's so wonderful! I was going to say..."],
            ["Linda", "she", "her",  "I thought steve was going to say..."],
            ["Raymond", "he", "his",  "Sorry. Linda, go ahead."],
            ["Linda", "she", "her",  "No, no, no, you go ahead."],
            ["Raymond", "he", "his",  "Ha, okay. Umm. I was saying... I had a good feeling about this."],
            ["Kai", "he", "his",  "Sorry, I'm in the grocery store. If you hear random noise, that's why."],
            ["Linda", "she", "her",  "We should totally celebrate next week when steve comes back."],
            ["Steve", "he", "his",  "Sure thing. The official news should come out next week. We will celebrate then."],
            ["Raymond", "he", "his",  "Awesome."],
            ["Kai", "he", "his",  "No. I don't want cashew. I meant almond."],
            ["Steve", "he", "his",  "Alright, enjoy your weekend, guys. Let's check in on Monday."],
            ["Kai", "he", "his",  "Sorry about that. Great job, team. So proud!"],
            ["Linda", "she", "her",  "Indeed. All the hard work paid off."],
            ["Raymond", "he", "his",  "Have a good weekend, everyone."],
            ["Kai", "he", "his",  "Same to you! Have a good one."],
            ["Linda", "she", "her",  "Goodbye, folks."],
            ["Steve", "he", "his",  "Talk to you later."],
        ],
        "epilogue" : "Raymond closes the call window and starts browsing"
    },
    {
        "title" : ["INT.","Union complex, Apartment 3C", "6:40PM"],
        "prologue" : "Raymond and Adrian sit down on the couch.",
        "dialogue" : [
            ["Adrian", "she", "her",  "That was so delicious."],
            ["Raymond", "he", "his",  "Netflix."],
            ["Adrian", "she", "her",  "and Chill?"],
            ["Raymond", "he", "his",  "Okay! Same old?"],
            ["Adrian", "she", "her",  "Yes, please."],
            ["Raymond", "he", "his",  "Let me see what episode we were on..."],
            ["Adrian", "she", "her",  "I think maybe episode 6?"],
            ["Raymond", "he", "his",  "O...okay. Beer?"],
            ["Adrian", "she", "her",  "Yes. I will take an IPA."],
            ["Raymond", "he", "his",  "of course. Here ya go."],
        ],
        "epilogue" : "Raymond presses the buttons on the remote, returns to the couch and hands Adrian the beer."
    },
    {
        "title" : ["INT.","Raymond’s bedroom", "11:40PM"],
        "prologue" : "Raymond puts his phone on his night stand and turns off the bedside lamp.",
        "dialogue" : [
            ["Raymond", "he", "his",  "Goodnight, world."],    
        ],
        "epilogue" : "Raymond closes his eyes."
    }

]

In [5]:
generate(scripts)

IndexError: list index out of range

In [2]:
print(scripts)

[{'title': ['INT.', 'Mud Spot', '7:00AM'], 'prologue': 'Raymond approaches the counter.', 'dialogue': [['Raymond', 'he', 'his', 'Small drip please.'], ['Barista', 'she', 'her', 'Cream or sugar?'], ['Raymond', 'he', 'his', 'Small drip please.'], ['Barista', 'she', 'her', 'Your name?'], ['Raymond', 'he', 'his', 'Raymond.']], 'epilogue': 'The barista nods, reaches for a cup, and begins his order.'}, {'title': ['EXT.', '2nd and 12th', '7:15AM'], 'prologue': 'Raymond looks right, and steps into the street.', 'dialogue': [['Bicyclist', 'he', 'his', 'Hey!']], 'epilogue': "Raymond steps back and sips his coffee. His jacket flaps once in the bicyclist's draft."}, {'title': ['INT.', 'Home', '10:00AM'], 'prologue': "Raymond's phone buzzes. He lets it go to voicemail.", 'dialogue': [["Raymond's mom", 'she', 'her', 'Hi honey. Give me a call back, okay?']], 'epilogue': 'She hangs up.'}, {'title': ['EXT.', 'Summersville Farmer’s Market', '10:10AM'], 'prologue': 'Raymond takes a sip of his coffee, wal